In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/number/images/N40.jpeg
/kaggle/input/number/images/N30.xml
/kaggle/input/number/images/N42.xml
/kaggle/input/number/images/N21.xml
/kaggle/input/number/images/N19.jpeg
/kaggle/input/number/images/N25.xml
/kaggle/input/number/images/N32.xml
/kaggle/input/number/images/N17.jpeg
/kaggle/input/number/images/N1.xml
/kaggle/input/number/images/N14.xml
/kaggle/input/number/images/N24.xml
/kaggle/input/number/images/N36.xml
/kaggle/input/number/images/N24.jpeg
/kaggle/input/number/images/N38.jpeg
/kaggle/input/number/images/N15.jpeg
/kaggle/input/number/images/N18.xml
/kaggle/input/number/images/N4.xml
/kaggle/input/number/images/N31.xml
/kaggle/input/number/images/N33.jpeg
/kaggle/input/number/images/N25.jpeg
/kaggle/input/number/images/N30.jpeg
/kaggle/input/number/images/N6.jpeg
/kaggle/input/number/images/N5.xml
/kaggle/input/number/images/N37.xml
/kaggle/input/number/images/N15.xml
/kaggle/input/number/images/N34.xml
/kaggle/input/number/images/N5.jpeg
/kaggle/input/number/i

In [82]:
import os
import cv2
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [83]:
path = glob('/kaggle/input/number/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:
    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [84]:
df = pd.DataFrame(labels_dict)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/kaggle/input/number/images/N30.xml,928,1219,543,606
1,/kaggle/input/number/images/N42.xml,163,373,253,327
2,/kaggle/input/number/images/N21.xml,244,403,221,258
3,/kaggle/input/number/images/N25.xml,132,235,169,193
4,/kaggle/input/number/images/N32.xml,360,500,171,206


In [85]:
image_path = list(df['filepath'].apply(getFilename))

In [86]:

def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/kaggle/input/number/images',filename_image)
    return filepath_image
getFilename(filename)

'/kaggle/input/number/images/N23.jpeg'

In [87]:
img_arr = cv2.imread(image_path[0])

In [88]:
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    img_arr=np.array(img_arr)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [89]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [90]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((29, 224, 224, 3), (8, 224, 224, 3), (29, 4), (8, 4))

In [97]:
model=VGG16(include_top=False,input_shape = (224,224,3),weights='imagenet')
model.trainable = False
flatten = Flatten()(model.output)
x= Dense(512 , activation = "relu")(flatten)
x= Dense(256 , activation = "relu")(x)
x= Dense(128 , activation = "relu")(x)
x= Dense(64 , activation = "relu")(x)
x= Dense(32 , activation = "relu")(x)
bbox = Dense(4)(x)
final=Model(inputs = model.input , outputs = bbox)
final.compile(loss='categorical_crossentropy', optimizer='adam')
final.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [98]:
final.fit(x_train,y_train,batch_size=8,epochs=100)

Epoch 1/10
4/4 [==============================] - 10s 2s/step - loss: 3.6386
Epoch 2/10
4/4 [==============================] - 8s 2s/step - loss: 3.0572
Epoch 3/10
4/4 [==============================] - 9s 2s/step - loss: 3.0106
Epoch 4/10
4/4 [==============================] - 8s 2s/step - loss: 3.0082
Epoch 5/10
4/4 [==============================] - 8s 2s/step - loss: 3.0041
Epoch 6/10
4/4 [==============================] - 8s 2s/step - loss: 3.0001
Epoch 7/10
4/4 [==============================] - 9s 2s/step - loss: 2.9972
Epoch 8/10
4/4 [==============================] - 8s 2s/step - loss: 2.9955
Epoch 9/10
4/4 [==============================] - 8s 2s/step - loss: 2.9922
Epoch 10/10
4/4 [==============================] - 8s 2s/step - loss: 2.9808
